# YOLOv8.1

## Test Cuda

In [1]:
import torch

In [2]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1060 6GB'

In [3]:
torch.cuda.device_count()

1

## Enable MLflow Logging

<!-- https://docs.ultralytics.com/integrations/mlflow/#setup-and-prerequisites -->

In [4]:
get_ipython().system_raw("mlflow ui --port 5000 --host 0.0.0.0 --backend-store-uri runs/mlflow &")
# http://localhost:5000/

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2024-02-28 16:08:44 +0000] [75] [INFO] Starting gunicorn 21.2.0
[2024-02-28 16:08:44 +0000] [75] [INFO] Listening at: http://0.0.0.0:5000 (75)
[2024-02-28 16:08:44 +0000] [75] [INFO] Using worker: sync
[2024-02-28 16:08:44 +0000] [76] [INFO] Booting worker with pid: 76
[2024-02-28 16:08:44 +0000] [77] [INFO] Booting worker with pid: 77
[2024-02-28 16:08:44 +0000] [78] [INFO] Booting worker with pid: 78
[2024-02-28 16:08:44 +0000] [79] [INFO] Booting worker with pid: 79


## Model Tuning

In [5]:
from ray import train, tune

### tune() Method Parameters

The `tune()` method in YOLOv8 provides an easy-to-use interface for hyperparameter tuning with [Ray Tune](https://docs.ray.io/en/latest/tune/index.html). It accepts several arguments that allow you to customize the tuning process. Below is a detailed explanation of each parameter:

| Parameter | Type | Description | Default Value |
|    --     |  --  |      --     |       --      |
| `data` | str | The dataset configuration file (in YAML format) to run the tuner on. This file should specify the training and validation data paths, as well as other dataset-specific settings. |  |
| `space` | dict, optional | A dictionary defining the hyperparameter search space for Ray Tune. Each key corresponds to a hyperparameter name, and the value specifies the range of values to explore during tuning. If not provided, YOLOv8 uses a default search space with various hyperparameters. |  |
| `grace_period` | int, optional | The grace period in epochs for the ASHA scheduler in Ray Tune. The scheduler will not terminate any trial before this number of epochs, allowing the model to have some minimum training before making a decision on early stopping. | 10 |
| `gpu_per_trial` | int, optional | The number of GPUs to allocate per trial during tuning. This helps manage GPU usage, particularly in multi-GPU environments. If not provided, the tuner will use all available GPUs. | None |
| `iterations` | int, optional | The maximum number of trials to run during tuning. This parameter helps control the total number of hyperparameter combinations tested, ensuring the tuning process does not run indefinitely. | 10 |
| `**train_args` | dict, optional | Additional arguments to pass to the train() method during tuning. These arguments can include settings like the number of training epochs, batch size, and other training-specific configurations. | {} |

In [ ]:
from ultralytics import YOLO

# Load a YOLOv8n model
model = YOLO('yolov8n.pt')

# Start tuning hyperparameters for YOLOv8n training on the COCO8 dataset
result_grid = model.tune(data='coco8.yaml', use_ray=True)

### Default Search Space Description

The following table lists the default search space parameters for hyperparameter tuning in YOLOv8 with Ray Tune. Each parameter has a specific value range defined by tune.uniform().

| Parameter | Value Range | Description |
|     --    |     --      |     --      |
| `lr0` | tune.uniform(1e-5, 1e-1) | Initial learning rate |
| `lrf` | tune.uniform(0.01, 1.0) | Final learning rate factor |
| `momentum` | tune.uniform(0.6, 0.98) | Momentum |
| `weight_decay` | tune.uniform(0.0, 0.001) | Weight decay |
| `warmup_epochs` | tune.uniform(0.0, 5.0) | Warmup epochs |
| `warmup_momentum` | tune.uniform(0.0, 0.95) | Warmup momentum |
| `box` | tune.uniform(0.02, 0.2) | Box loss weight |
| `cls` | tune.uniform(0.2, 4.0) | Class loss weight |
| `hsv_h` | tune.uniform(0.0, 0.1) | Hue augmentation range |
| `hsv_s` | tune.uniform(0.0, 0.9) | Saturation augmentation range |
| `hsv_v` | tune.uniform(0.0, 0.9) | Value (brightness) augmentation range |
| `degrees` | tune.uniform(0.0, 45.0) | Rotation augmentation range (degrees) |
| `translate` | tune.uniform(0.0, 0.9) | Translation augmentation range |
| `scale` | tune.uniform(0.0, 0.9) | Scaling augmentation range |
| `shear` | tune.uniform(0.0, 10.0) | Shear augmentation range (degrees) |
| `perspective` | tune.uniform(0.0, 0.001) | Perspective augmentation range |
| `flipud` | tune.uniform(0.0, 1.0) | Vertical flip augmentation probability |
| `fliplr` | tune.uniform(0.0, 1.0) | Horizontal flip augmentation probability |
| `mosaic` | tune.uniform(0.0, 1.0) | Mosaic augmentation probability |
| `mixup` | tune.uniform(0.0, 1.0) | Mixup augmentation probability |
| `copy_paste` | tune.uniform(0.0, 1.0) | Copy-paste augmentation probability |


In this example, we demonstrate how to use a custom search space for hyperparameter tuning with Ray Tune and YOLOv8. By providing a custom search space, you can focus the tuning process on specific hyperparameters of interest:

In [ ]:
# Load your model
model = YOLO("path/to/your/model.pt")

# Define the hyperparameter search space
space = {
    "lr0": tune.uniform(1e-5, 1e-1),
    "weight_decay": tune.uniform(0.0, 0.001),
    "warmup_epochs": tune.uniform(0.0, 5.0),
    "box": tune.uniform(0.02, 0.2),
    "cls": tune.uniform(0.2, 4.0)
}

# Define additional training arguments
train_args = {"epochs": 100}

# Start the hyperparameter tuning process
result = model.tune(
    data="path/to/your/data.yaml",
    space=space,
    train_args=train_args,
    gpu_per_trial=1,
    max_samples=100
)

In [ ]:
from ultralytics import YOLO

# Define a YOLO model
model = YOLO("yolov8n.pt")

# Run Ray Tune on the model
result_grid = model.tune(data="coco128.yaml",
                         space={"lr0": tune.uniform(1e-5, 1e-1)},
                         epochs=10,
                         use_ray=True)

(_tune pid=599) Ultralytics YOLOv8.1.19 🚀 Python-3.10.13 torch-2.2.1 CPU (Intel Core(TM) i7-7700 3.60GHz)
(_tune pid=599) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

  0%|          | 0.00/6.66M [00:00<?, ?B/s]
  0%|          | 8.00k/6.66M [00:00<01:37, 71.4kB/s]
  0%|          | 32.0k/6.66M [00:00<00:44, 158kB/s] 
  1%|          | 56.0k/6.66M [00:00<00:46, 150kB/s]
  2%|▏         | 136k/6.66M [00:00<00:22, 307kB/s] 
  4%|▍         | 280k/6.66M [00:00<00:10, 622kB/s]
  7%|▋         | 488k/6.66M [00:00<00:06, 1.04MB/s]
 10%|▉         | 648k/6.66M [00:00<00:05, 1.21MB/s]
 15%|█▍        | 0.99M/6.66M [00:00<00:03, 1.93MB/s]
 21%|██        | 1.41M/6.66M [00:01<00:02, 2.46MB/s]
 34%|███▍      | 2.27M/6.66M [00:01<00:01, 4.11MB/s]
 41%|████      | 2.74M/6.66M [00:01<00:00, 4.20MB/s]
 47%|████▋     | 3.15M/6.66M [00:01<00:01, 3.14MB/s]
 52%|█████▏    | 3.49M/6.66M [00:01<00:01, 2.71MB/s]
 57%|█████▋    | 3.79M/6.66M [00:01<00:01, 2.19MB/s]
 61%|██████    | 4.03M/6.66M [00:02<00:01, 2.18MB/s]
 64%|██████▍   | 4.27M/6.66M [00:02<00:01, 1.68MB/s]
 67%|██████▋   | 4.46M/6.66M [00:02<00:01, 1.34MB/s]
 69%|██████▉   | 4.62M/6.66M [00:02<00:02, 1.07MB/s]
 71%|███

(_tune pid=599) Dataset download success ✅ (10.6s), saved to /opt/app/notebooks/datasets
(_tune pid=599) 
(_tune pid=599) Downloading https://ultralytics.com/assets/Arial.ttf to '/root/.config/Ultralytics/Arial.ttf'...


  0%|          | 0.00/755k [00:00<?, ?B/s]
  3%|▎         | 24.0k/755k [00:00<00:03, 219kB/s]
 18%|█▊        | 136k/755k [00:00<00:01, 534kB/s] 
 34%|███▍      | 256k/755k [00:00<00:00, 657kB/s]
 64%|██████▎   | 480k/755k [00:00<00:00, 1.11MB/s]
100%|██████████| 755k/755k [00:00<00:00, 1.34MB/s]


(_tune pid=599) 
(_tune pid=599)                    from  n    params  module                                       arguments                     
(_tune pid=599)   0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
(_tune pid=599)   1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
(_tune pid=599)   2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
(_tune pid=599)   3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
(_tune pid=599)   4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
(_tune pid=599)   5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
(_tune pid=599)   6                  -1  2    197632  ultralytics.nn.modu

...

       6/10         0G      1.168      1.324      1.168        103        640:  12%|█▎        | 1/8 [00:04<00:34,  4.97s/it]
       6/10         0G      1.126      1.215       1.16        149        640:  25%|██▌       | 2/8 [00:09<00:28,  4.75s/it]
       6/10         0G      1.179      1.227      1.181        115        640:  38%|███▊      | 3/8 [00:14<00:24,  4.91s/it]
       6/10         0G      1.135       1.16      1.155         93        640:  50%|█████     | 4/8 [00:19<00:19,  4.85s/it]
       6/10         0G      1.189      1.235       1.18        186        640:  62%|██████▎   | 5/8 [00:24<00:14,  4.85s/it]
       6/10         0G      1.147      1.251      1.182         53        640:  75%|███████▌  | 6/8 [00:29<00:09,  4.81s/it]
       6/10         0G       1.13      1.242       1.17         79        640:  88%|████████▊ | 7/8 [00:33<00:04,  4.81s/it]
       6/10         0G      1.105      1.209      1.162         95        640: 100%|██████████| 8/8 [00:38<00:00,  4.79s/it]


(_tune pid=1226)                    all        128        929      0.693      0.625      0.683      0.511


  0%|          | 0/8 [00:00<?, ?it/s]


(_tune pid=1226) 
(_tune pid=1226)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/10         0G      1.223      1.294      1.211         91        640:  12%|█▎        | 1/8 [00:05<00:38,  5.53s/it]
       7/10         0G      1.192      1.206      1.186        132        640:  25%|██▌       | 2/8 [00:10<00:30,  5.08s/it]


### Processing Ray Tune Results

In [ ]:
# storage_path = 'runs/detect/tune'
# tune = '_tune_2024-02-27_17-08-37'
# experiment = 'experiment_state-2024-02-27_17-08-37.json'

# experiment_path = f"{storage_path}/{tune}/{experiment}"
# print(f"Loading results from {experiment_path}...")

# restored_tuner = tune.Tuner.restore(experiment_path, trainable="_tune")
# result_grid = restored_tuner.get_results()

In [21]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.022139359677241066, 'data': 'coco128.yaml', 'epochs': 10}, Last Reported Metrics: {'metrics/precision(B)': 0.6870841651209729, 'metrics/recall(B)': 0.6770751611424184, 'metrics/mAP50(B)': 0.7106112928649144, 'metrics/mAP50-95(B)': 0.5297666352827185, 'fitness': 0.5478511010409381, 'timestamp': 1709054308, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': 'd8cbe_00000', 'date': '2024-02-27_17-18-28', 'time_this_iter_s': 589.4447066783905, 'time_total_s': 589.4447066783905, 'pid': 1041, 'hostname': 'fec87aab8937', 'node_ip': '172.17.0.2', 'config': {'lr0': 0.022139359677241066, 'data': 'coco128.yaml', 'epochs': 10}, 'time_since_restore': 589.4447066783905, 'iterations_since_restore': 1, 'experiment_tag': '0_lr0=0.0221'}
Trial #1: Configuration: {'lr0': 0.07427690062071145, 'data': 'coco128.yaml'}, Last Reported Metrics: {'trial_id': 'd8cbe_00001', 'date': '2024-02-27_17-18-28', 'timestamp': 1709054308, 'pid': 1041, 'hostnam

In [31]:
for result in result_grid:
    print(result)

Result(
  metrics={'metrics/precision(B)': 0.6870841651209729, 'metrics/recall(B)': 0.6770751611424184, 'metrics/mAP50(B)': 0.7106112928649144, 'metrics/mAP50-95(B)': 0.5297666352827185, 'fitness': 0.5478511010409381},
  path='/opt/app/notebooks/runs/detect/tune/_tune_2024-02-27_17-08-37/_tune_d8cbe_00000_0_lr0=0.0221_2024-02-27_17-08-37',
  filesystem='local',
  checkpoint=None
)
Result(
  error='RayTaskError',
  metrics={},
  path='/opt/app/notebooks/runs/detect/tune/_tune_2024-02-27_17-08-37/_tune_d8cbe_00001_1_lr0=0.0743_2024-02-27_17-08-37',
  filesystem='local',
  checkpoint=None
)
Result(
  metrics={'metrics/precision(B)': 0.6870841651209729, 'metrics/recall(B)': 0.6770751611424184, 'metrics/mAP50(B)': 0.7106112928649144, 'metrics/mAP50-95(B)': 0.5297666352827185, 'fitness': 0.5478511010409381},
  path='/opt/app/notebooks/runs/detect/tune/_tune_2024-02-27_17-08-37/_tune_d8cbe_00002_2_lr0=0.0014_2024-02-27_17-08-37',
  filesystem='local',
  checkpoint=None
)
Result(
  error='RayT

In [39]:
for result in result_grid:
    if not result_grid.errors:
      print(result.metrics_dataframe["metrics/precision(B)"])

In [ ]:
import matplotlib.pyplot as plt

for result in result_grid:
    plt.plot(result.metrics_dataframe["training_iteration"], result.metrics_dataframe["metrics/mAP50(B)"], label=f"Trial {i}")

plt.xlabel('Training Iterations')
plt.ylabel('Mean Accuracy')
plt.legend()
plt.show()

In [ ]:
from ultralytics import YOLO
from ultralytics.utils.tuner import run_ray_tune

model = YOLO("model_path")

run_ray_tune(model, data="ymal_file_path", grace_period=30, gpu_per_trial=1)

In [4]:
from ultralytics import settings

# Update a setting
settings.update({
    'mlflow': True,
    'clearml': False,
    'comet': False,
    'dvc': False,
    'hub': False,
    'neptune': False,
    'raytune': False,
    'tensorboard': False,
    'wandb': False
})

# Reset settings to default values
# settings.reset()

In [5]:
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': '/opt/app/notebooks/datasets', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': 'af86e989f891cbd0cd78de981596cc553f47f4a708b5ec210405ae2024cb9aec', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': False, 'comet': False, 'dvc': False, 'hub': False, 'mlflow': True, 'neptune': False, 'raytune': False, 'tensorboard': False, 'wandb': False}


## Model Training

In [7]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
# model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

# Train the model using the 'coco128.yaml' dataset for 3 epochs
results = model.train(data='coco128.yaml', epochs=3)

New https://pypi.org/project/ultralytics/8.1.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.17 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6070MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

train: Scanning /opt/app/notebooks/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 
val: Scanning /opt/app/notebooks/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 10


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
MLflow: logging run_id(990096c31fbc48c59c97834ec396bd4f) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3       2.6G      1.226      1.615      1.274        178        640: 100%|██████████| 8/8 [00:03<00:00,  2
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:0

                   all        128        929      0.644      0.527      0.612      0.454


...

        3/3      2.65G      1.207      1.442      1.221        178        640: 100%|██████████| 8/8 [00:03<00:00,  2
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:0

                   all        128        929      0.671      0.547      0.629      0.468



3 epochs completed in 0.005 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.17 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6070MiB)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:0


                   all        128        929      0.672      0.547       0.63      0.469
                person        128        254       0.78      0.677      0.767       0.54
               bicycle        128          6      0.601      0.333      0.325      0.281
                   car        128         46      0.798      0.217      0.281      0.185
            motorcycle        128          5      0.685      0.877      0.938      0.721
              airplane        128          6      0.828      0.806      0.903      0.675
                   bus        128          7      0.717      0.714      0.736      0.648
                 train        128          3      0.551      0.667      0.723      0.645
                 truck        128         12          1      0.376      0.491       0.31
                  boat        128          6      0.284      0.167      0.376      0.229
         traffic light        128         14      0.712      0.181      0.202      0.139
             stop sig

In [8]:
# Evaluate the model's performance on the validation set
results = model.val()

Ultralytics YOLOv8.1.17 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6070MiB)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /opt/app/notebooks/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 10
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:0


                   all        128        929      0.669      0.547      0.627      0.467
                person        128        254      0.786      0.681      0.766      0.542
               bicycle        128          6      0.595      0.333      0.323       0.28
                   car        128         46      0.786      0.217      0.284      0.183
            motorcycle        128          5      0.687      0.884      0.898       0.73
              airplane        128          6      0.828      0.808      0.903      0.674
                   bus        128          7      0.668      0.714      0.735      0.648
                 train        128          3      0.548      0.667      0.731      0.652
                 truck        128         12      0.986      0.417      0.497      0.299
                  boat        128          6      0.273      0.167      0.334      0.142
         traffic light        128         14      0.702      0.173      0.202      0.139
             stop sig

In [9]:
# Perform object detection on an image using the model
results = model('https://ultralytics.com/images/bus.jpg')


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /opt/app/notebooks/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 9.3ms
Speed: 3.1ms preprocess, 9.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)


In [10]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

## Gradio for Object Detection

In [5]:
get_ipython().system_raw("python serve.py &")
# http://localhost:7861/

Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.

0: 384x640 1 person, 1 sports ball, 84.4ms
Speed: 4.3ms preprocess, 84.4ms inference, 295.8ms postprocess per image at shape (1, 3, 384, 640)
